In [1]:
from youtube_dl import YoutubeDL

import moviepy.editor as mp

import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
import nltk
from sumy.summarizers.lsa import LsaSummarizer

from deep_translator import GoogleTranslator
from bs4 import BeautifulSoup
import requests

import IPython
import noisereduce as nr
import soundfile as sf
from noisereduce.generate_noise import band_limited_noise
import urllib.request
import io

import os

ModuleNotFoundError: No module named 'pytube'

In [7]:
def youtubeLink_video(video_link, save_path='extracted_video.mp4'):
    try: 
        with YoutubeDL({'format':'bestaudio', 'outtmpl': 'audio'}) as ydl:
            audio = ydl.extract_info(video_link)
    except: 
        print("Sry servers are full now!") 

In [8]:
youtubeLink_video("https://www.youtube.com/watch?v=vP4iY1TtS3s")

In [11]:
def audio_extraction(video_path, save_path="extracted_audio.wav"):
    clip = mp.VideoFileClip(video_path)
    clip.audio.write_audiofile(save_path)

In [13]:
audio_extraction('C:/Users/Nimisha/Desktop/Programs/WE/we-project1/nlp-project/extracted_video.mp4/I Have a Dream speech by Martin Luther King Jr HD (subtitled).mp4')

MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.


In [ ]:
# speech to text
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(),"speechtotext-329110-21d3821f64eb.json")

def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    import io

    client = speech.SpeechClient()

    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="en-US",
    )

    response = client.recognize(config=config, audio=audio)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    f = open(speech_file+".txt", "w")
    for result in response.results:
        f.write(result.alternatives[0].transcript + "\n")
        # The first alternative is the most likely one for this portion.
        print(u"{}".format(result.alternatives[0].transcript))
    f.close()

In [ ]:
def text_summarization(PATH):
    nltk.download('punkt')
    with open(PATH, 'r', encoding="utf8") as file:
        text = file.read().splitlines()
        no_of_lines = len(text) // 3
        text = ' '.join(text)

    my_parser = PlaintextParser.from_string(text, Tokenizer('english'))
    
    lsa_summarizer = LsaSummarizer()
    lsa_summary = lsa_summarizer(my_parser.document, no_of_lines)

    with open("Output.txt", "w") as text_file:
        for sentence in lsa_summary:
            print(sentence, file=text_file)

In [3]:
# Languages for translation
url='https://developers.google.com/admin-sdk/directory/v1/languages'

html_content = requests.get(url).text

soup = BeautifulSoup(html_content, "lxml")

s = list(filter(('').__ne__, soup.table.text.split('\n')))

code = [i for j, i in enumerate(s) if j % 2 != 0][1:]
name = [i for j, i in enumerate(s) if j % 2 == 0][1:]

languages = dict(zip(name, code))

In [ ]:
def text_translation(text, language='tamil'):
    with open(text, encoding="utf8") as f:
        lines = f.read().splitlines()
    
    lines = ' '.join(lines)

    translated_text = GoogleTranslator(source='auto', target=language).translate(lines)
    return translated_text  

In [ ]:
def audio_encoding(url):
    response = urllib.request.urlopen(url)
    data, rate = sf.read(io.BytesIO(response.read()))
    return data, rate
    
def add_noise(data, rate):
    noise = band_limited_noise(min_freq = 2000, max_freq = 12000, samples = len(data), samplerate = rate) * 10
    return data + noise

def audio_without_noise(data, rate):
    reduced_noise = nr.reduce_noise(y = add_noise(), sr = rate, n_std_thresh_stationary = 1.5, stationary = True)
    return reduced_noise

def background_noice_removal():
    data, rate = audio_encoding(url)
    IPython.display.Audio(data = audio_without_noise(data, rate), rate = rate)